In [24]:
import numpy as np
import pandas as pd



# Import danych

In [25]:
train_file_path = "../data/iris_trn.txt"
test_file_path = "../data/iris_tst.txt"

train_data = pd.read_csv(train_file_path, sep="  ", skiprows=1, header=0)

column_name = ["Class"]

for index in range(1, len(train_data.columns)):
    column_name.append(f"P{index}")

train_data.columns = column_name
train_data

/opt/conda/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,Class,P1,P2,P3,P4
0,1,4.9,3.0,1.4,0.2
1,1,4.7,3.2,1.3,0.2
2,1,4.6,3.1,1.5,0.2
3,1,5.0,3.6,1.4,0.2
4,1,5.4,3.9,1.7,0.4
...,...,...,...,...,...
69,3,6.9,3.2,5.7,2.3
70,3,5.6,2.8,4.9,2.0
71,3,7.7,2.8,6.7,2.0
72,3,6.3,2.7,4.9,1.8


In [43]:
gravity_centers = train_data.groupby(train_data.Class).mean()
gravity_centers

,P1,P2,P3,P4
Class,,,,
1,5.025,3.479167,1.4625,0.250
2,6.012,2.776000,4.3120,1.344
3,6.576,2.928000,5.6400,2.044


In [44]:
train_data_no_class = train_data.drop(columns=["Class"])

In [45]:
mv = train_data_no_class.mean()
mv

P1    5.882432
P2    3.055405
P3    3.836486
P4    1.225676
dtype: float64

In [46]:
sd = train_data_no_class.std()
sd

P1    0.857898
P2    0.468791
P3    1.805287
P4    0.765535
dtype: float64

In [39]:
gravity_centers_std = (gravity_centers-mv)/sd
gravity_centers_std

,P1,P2,P3,P4
Class,,,,
1,-0.999458,0.903945,-1.315019,-1.274501
2,0.151029,-0.596013,0.263400,0.154564
3,0.808450,-0.271774,0.999018,1.068957


In [50]:
train_data_std = (train_data_no_class-mv)/sd
train_data_std

,P1,P2,P3,P4
0,-1.145163,-0.118188,-1.349639,-1.339815
1,-1.378291,0.308442,-1.405032,-1.339815
2,-1.494855,0.095127,-1.294246,-1.339815
3,-1.028599,1.161700,-1.349639,-1.339815
4,-0.562343,1.801645,-1.183461,-1.078560
...,...,...,...,...
69,1.186118,0.308442,1.032253,1.403364
70,-0.329215,-0.544817,0.589110,1.011481
71,2.118630,-0.544817,1.586182,1.011481
72,0.486734,-0.758132,0.589110,0.750226


In [66]:
ds = (train_data_std-gravity_centers_std)
ds

,P1,P2,P3,P4
0,NaN,NaN,NaN,NaN
1,-0.378833,-0.595504,-0.090013,-0.065314
2,-1.645884,0.691140,-1.557647,-1.494379
3,-1.837049,1.433475,-2.348657,-2.408772
4,NaN,NaN,NaN,NaN
...,...,...,...,...
69,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN


In [92]:
distances = pd.DataFrame()
for index, gravity_center_std in enumerate(gravity_centers_std.values):
    data = []
    for train_data_value in train_data_std.values:
        data.append( 
            np.linalg.norm(gravity_center_std-train_data_value)**2
        )
    distances[f"D{index+1}"] = data
distances

,D1,D2,D3
0,1.071450,6.743497,15.158566
1,0.510507,8.173695,16.700126
2,0.904303,7.846042,16.501075
3,0.072751,9.316145,16.747973
4,1.052634,9.871664,15.553182
...,...,...,...
69,17.812012,4.040081,0.592216
70,11.399563,1.073649,1.540161
71,25.464062,6.358130,2.139188
72,12.696494,0.599880,0.609659


In [87]:
np.linalg.norm(gravity_centers_std.values[2]-train_data_std.values[0])

3.89340032422192

1